# 패키지불러오기

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.14.0


In [ ]:
!pip install colormath
!pip install tensorflow-addons


  Preparing metadata (setup.py) ... done
  Created wheel for colormath: filename=colormath-3.0.0-py3-none-any.whl size=39405 sha256=e1956abf8c4fa95f79ab55d7043a8f812cb4a0601ac7d371735c4b1a5e37b8eb
  Stored in directory: /root/.cache/pip/wheels/ab/b3/4d/c0738759c25a1df01958068f162cf2a9dc3ab1da8b972cfcfc
Successfully built colormath
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
import asyncio
import datetime
import logging
import traceback

import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.metrics import r2_score
import pickle
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from colormath.color_objects import LabColor, sRGBColor, XYZColor, LabColor
from colormath.color_diff import delta_e_cmc
from colormath.color_conversions import convert_color

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors

from PIL import Image
import math
from math import sqrt
import copy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
import pandas.core.algorithms as algos
from pandas import Series
import pickle
#from pandas import tools

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")

In [ ]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cmc
import numpy as np
import pandas as pd
from colormath.color_objects import sRGBColor, XYZColor, LabColor
from colormath.color_conversions import convert_color
import pandas as pd
import math
import numpy as np
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from PIL import Image
from matplotlib import pyplot as plt
import copy
from sklearn.preprocessing import LabelEncoder
import pandas.core.algorithms as algos
from pandas import Series
import pickle
#from pandas import tools

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.linear_model import LinearRegression
from tensorflow import keras
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
# gpu 상태 확인 및 메모리 사용만큼 할당
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
import webcolors
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
from colormath import color_diff_matrix
import matplotlib.colors as mcolors

from math import sqrt
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 필요함수

In [ ]:
def paste_python(aa,a):
    for i in range(len(aa)):
        aa[i]=a+aa[i]
    return(aa)

In [ ]:
def paste_python2(a,aa):
    for i in range(len(aa)):
        aa[i]=aa[i]+a
    return(aa)

In [ ]:
# lab를 rgb로 변환하는 함수
def lab_to_rgb(data):
    r_list=[]
    g_list=[]
    b_list=[]
    for i in range(len(data)):
        lab=LabColor(data[data.columns[0]][data.index[i]],data[data.columns[1]][data.index[i]],data[data.columns[2]][data.index[i]])#lab좌표
        rgb=convert_color(lab,sRGBColor)#rgb좌표
        rgb_list=[color for color in rgb.get_value_tuple()]
        r_list.append(rgb_list[0])
        g_list.append(rgb_list[1])
        b_list.append(rgb_list[2])
    df = pd.DataFrame(
                {'r' : r_list,
                 'g' : g_list,
                 'b' : b_list}
                )
    df.index=data.index
    return(df)

In [ ]:
# CMC 색차를 계산합니다.
def cmc_color_difference(Lab1, Lab2, l=1, c=1):

    # CMC 색차를 계산합니다.
    SL = 0.04079 * Lab1[0] + 0.968 * Lab1[0] / 100
    SC = 0.06377 * Lab1[1] + 0.839 * Lab1[1] / 100
    SH = 0.07275 * Lab1[2] + 1.037 * Lab1[2] / 100

    delta_L = Lab2[0] - Lab1[0]
    delta_C = Lab2[1] - Lab1[1]
    delta_H = Lab2[2] - Lab1[2]

    delta_E = ((delta_L / (l * SL)) ** 2 + (delta_C / (c * SC)) ** 2 + (delta_H / SH) ** 2) ** 0.5

    return delta_E

In [ ]:
def cie94_color_difference(y_pred, y_true):
    """Calculate color difference by using CIE94 formulae

    See http://en.wikipedia.org/wiki/Color_difference or
    http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

    cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
    >>> 58.0
    cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
    >>> 0.0
    """

    L1, a1, b1 = y_true[0],y_true[1],y_true[2]
    L2, a2, b2 = y_pred[0],y_pred[1],y_pred[2]


    C1 = np.sqrt(a1**2 + b1**2)
    C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
    delta_L = L1 - L2
    delta_C = C1 - C2
    delta_a = a1 - a2
    delta_b = b1 - b2
    delta_H_square = delta_a**2 + delta_b**2 - delta_C**2
    loss_a = (np.sqrt((delta_L)**2
            + (delta_C)**2 / (1.0 + 0.045 * C1)**2
            + delta_H_square / (1.0 + 0.015 * C1)**2))
    return loss_a

In [ ]:
def color_diff(pred_TRANS,Y_TRANS):
    diff = []
    for i in Y_TRANS.index:
        pred = pred_TRANS.loc[i]
        true = Y_TRANS.loc[i]
        p_lab = LabColor(pred[0],pred[1],pred[2])
        t_lab = LabColor(true[0],true[1],true[2])
        delta = delta_e_cmc(p_lab , t_lab)
        diff.append(delta)
    diff=pd.DataFrame(diff)
    diff.index=Y_TRANS.index
    return(diff)

In [ ]:
def color_diff(pred_TRANS,Y_TRANS):
    diff = []
    for i in Y_TRANS.index:
        pred = list(pred_TRANS.loc[i])
        true = list(Y_TRANS.loc[i])
#        p_lab = LabColor(pred[0],pred[1],pred[2])
#        t_lab = LabColor(true[0],true[1],true[2])
        delta = cie94_color_difference(pred , true)
        diff.append(delta)
    diff=pd.DataFrame(diff)
    diff.index=Y_TRANS.index
    return(diff)

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
    # 첨 class 선언신 변수생성
    def __init__(self,input_units, num_unit,output_unit,lams,**kwargs,
                ):
        super(AttentionLayer, self).__init__(**kwargs)
        self.input_units = input_units
        self.num_unit = num_unit
        self.output_unit = output_unit
        self.lambdas = lams

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'input_units': self.input_units,
            'num_unit': self.num_unit,
            'output_unit': self.output_unit,
            'lambdas': self.lambdas,
        })
        return config

    def build(self, input_shape):
        self.kernel1 = self.add_weight("kernel1",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape =[self.input_units,self.num_unit])
        self.bias1 = self.add_weight("bias1",initializer = tf.zeros_initializer(),shape=[1,self.num_unit])



        self.kernel2 = self.add_weight("kernel2",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape = [self.num_unit,self.output_unit])
        self.bias2 = self.add_weight("bias2",initializer = tf.zeros_initializer(),shape=[1,self.output_unit])



    def call(self, inputs):
        shape = [ tf.shape(inputs)[k] for k in range(len(inputs.shape))]

        shape_list = [shape[0]] + [1 for i in range(len(inputs.shape)-1)]
        kernel1_ = tf.tile(tf.expand_dims(self.kernel1,0),shape_list)
        kernel2_ = tf.tile(tf.expand_dims(self.kernel2,0),shape_list)

        h1 = tf.matmul(inputs,kernel1_) + self.bias1
        h1 = tf.nn.tanh(h1)
        out = tf.matmul(h1,kernel2_) + self.bias2
        out = tf.nn.softmax(out,axis=1)
        flat_output_total = tf.multiply(inputs,tf.expand_dims(out[:,:,0],axis=2))
        flat_output_total = tf.reduce_sum(flat_output_total,axis=1)
        for i in range(1,self.output_unit):
            flat_output= tf.multiply(inputs,tf.expand_dims(out[:,:,i],axis=2))
            flat_output = tf.reduce_sum(flat_output,axis=1)
            flat_output_total = tf.concat([flat_output_total,flat_output],axis=1)

        return flat_output_total

In [ ]:
    # CIE94 Keras Custom Loss
    def cie94(y_true, y_pred):
        """Calculate color difference by using CIE94 formulae

        See http://en.wikipedia.org/wiki/Color_difference or
        http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

        cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
        >>> 58.0
        cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
        >>> 0.0
        """

        L1, a1, b1 = y_true[:,0],y_true[:,1],y_true[:,2]
        L2, a2, b2 = y_pred[:,0],y_pred[:,1],y_pred[:,2]


        C1 = K.sqrt(K.pow(a1,2) + K.pow(b1,2))
        C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
        delta_L = L1 - L2
        delta_C = C1 - C2
        delta_a = a1 - a2
        delta_b = b1 - b2
        delta_H_square = K.pow(delta_a,2) + K.pow(delta_b,2) - K.pow(delta_C,2)
        loss_a = (K.sqrt(K.pow(delta_L,2)
                + K.pow(delta_C,2) / K.pow(1.0 + 0.045 * C1,2)
                + delta_H_square / K.pow(1.0 + 0.015 * C1,2)))
        return loss_a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터불러오기

In [ ]:
#데이터 불러오기
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/기초데이터/train_data.csv',encoding='euc-kr',index_col=0)
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/기초데이터/test_data.csv',encoding='euc-kr',index_col=0)

#shape 확인
print('train shape : ',train.shape)
print('test shape : ',test.shape)

#인덱스정보 따로 저장
# train_index=train['index']
# test_index=test['index']

#데이터 결합
data=pd.concat([train,test],axis=0)
data


train shape :  (3347, 174)
test shape :  (372, 174)


,index,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,...,배합_염료3_Black_prop,배합_염료1_Grey_prop,배합_염료2_Grey_prop,배합_염료3_Grey_prop,배합_염료2_Brown_prop,배합_염료1_Brown_prop,배합_염료3_Brown_prop,색상_L*,색상_a*,색상_b*
1748,1748,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,0,20.7300,8.2000,-20.2100
3730,3730,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,...,0,0,0,0,0,0,0,72.4890,25.1515,84.0940
308,308,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,0,50.1900,6.8500,-20.2500
930,930,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,0,50.6250,62.5350,17.1050
49,49,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,1,0,0,57.1100,34.0900,55.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,1094,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,1,54.2400,9.0675,9.6225
1895,1895,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,0,65.3550,43.9665,22.5465
1020,1020,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,0,34.3850,-18.0250,1.6150
352,352,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,...,0,0,0,0,0,0,0,45.4500,-15.8800,25.3500


In [ ]:
train.shape

(3347, 174)

In [ ]:
# 이상치로 판단된 인덱스 리스트
# outlier_indices = [1403, 1352, 1347, 1294, 3485, 1028, 1103, 1104, 516, 450, 373, 2917, 2272, 2484, 2190, 2106]
# train_out=[1403, 1347, 1294, 1103, 1104, 516, 450, 373, 2917, 2272, 2484, 2190, 2106]
# test_out=[1352, 3485, 1028]
# DataFrame에서 이상치 인덱스를 제거
# data.drop(index=outlier_indices, inplace=True)
# train.drop(index=train_out, inplace=True)
# test.drop(index=test_out, inplace=True)
# 이제 'data' DataFrame은 이상치 행을 제외한 데이터를 포함합니다.
train_index=train['index']
test_index=test['index']

#index 제거
data=data[data.columns[1:]]
data

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop,배합_염료1_Grey_prop,배합_염료2_Grey_prop,배합_염료3_Grey_prop,배합_염료2_Brown_prop,배합_염료1_Brown_prop,배합_염료3_Brown_prop,색상_L*,색상_a*,색상_b*
1748,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,20.7300,8.2000,-20.2100
3730,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,72.4890,25.1515,84.0940
308,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,50.1900,6.8500,-20.2500
930,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,50.6250,62.5350,17.1050
49,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,1,0,0,57.1100,34.0900,55.9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,1,54.2400,9.0675,9.6225
1895,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,65.3550,43.9665,22.5465
1020,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,34.3850,-18.0250,1.6150
352,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,45.4500,-15.8800,25.3500


In [ ]:
_df = data[['배합_염료1_Yellow',
           '배합_염료1_Red', '배합_염료1_Blue', '배합_염료2_Yellow', '배합_염료2_Red',
           '배합_염료2_Blue', '배합_염료3_Yellow', '배합_염료3_Red', '배합_염료3_Blue',
           '배합_염료1_Black', '배합_염료2_Black', '배합_염료3_Black', '배합_염료1_Grey',
           '배합_염료2_Grey', '배합_염료3_Grey', '배합_염료2_Brown', '배합_염료1_Brown',
           '배합_염료3_Brown',]]
_df

,배합_염료1_Yellow,배합_염료1_Red,배합_염료1_Blue,배합_염료2_Yellow,배합_염료2_Red,배합_염료2_Blue,배합_염료3_Yellow,배합_염료3_Red,배합_염료3_Blue,배합_염료1_Black,배합_염료2_Black,배합_염료3_Black,배합_염료1_Grey,배합_염료2_Grey,배합_염료3_Grey,배합_염료2_Brown,배합_염료1_Brown,배합_염료3_Brown
1748,0.000,2.000,4.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3730,3.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,0.000,0.167,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,0.000,2.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1895,0.167,0.333,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1020,1.000,0.000,2.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,1.000,0.000,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
target_word = ["염료1", "염료2", "염료3",
                   "Yellow", "Red", "Blue", "Black", "Grey", "Brown",
                   ]

for index, row in _df.iterrows():
        # Iterate over each element in the row
        for col_name, value in row.items():
            # Check if the element is not equal to 0
            if value != 0:
                # Check if any target word is a substring of the column name
                for word in target_word:
                    if word in col_name:
                        # Assign 1 to the column named with the target word
                        _df.at[index, word] = 1

In [ ]:
_df = _df.fillna(0)
_df

,배합_염료1_Yellow,배합_염료1_Red,배합_염료1_Blue,배합_염료2_Yellow,배합_염료2_Red,배합_염료2_Blue,배합_염료3_Yellow,배합_염료3_Red,배합_염료3_Blue,배합_염료1_Black,...,배합_염료3_Brown,염료1,Red,Blue,Yellow,염료2,Brown,염료3,Black,Grey
1748,0.000,2.000,4.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3730,3.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
308,0.000,0.167,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
930,0.000,2.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1895,0.167,0.333,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1020,1.000,0.000,2.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
352,1.000,0.000,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
_df2 = _df[_df.columns[:18]]
_df['염료_total'] = _df2.sum(axis=1)
_df = _df.drop(columns=['배합_염료1_Yellow', '배합_염료1_Red', '배합_염료1_Blue', '배합_염료2_Yellow',
    '배합_염료2_Red', '배합_염료2_Blue', '배합_염료3_Yellow', '배합_염료3_Red',
    '배합_염료3_Blue', '배합_염료1_Black', '배합_염료2_Black', '배합_염료3_Black',
    '배합_염료1_Grey', '배합_염료2_Grey', '배합_염료3_Grey', '배합_염료2_Brown',
    '배합_염료1_Brown', '배합_염료3_Brown'])

In [ ]:
data = pd.concat([data, _df], axis=1)
data = data.drop(columns=['배합_염료1_Yellow', '배합_염료1_Red', '배합_염료1_Blue', '배합_염료2_Yellow',
         '배합_염료2_Red', '배합_염료2_Blue', '배합_염료3_Yellow', '배합_염료3_Red',
          '배합_염료3_Blue', '배합_염료1_Black', '배합_염료2_Black', '배합_염료3_Black',
           '배합_염료1_Grey', '배합_염료2_Grey', '배합_염료3_Grey', '배합_염료2_Brown',
           '배합_염료1_Brown', '배합_염료3_Brown'])

In [ ]:
data

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,염료1,Red,Blue,Yellow,염료2,Brown,염료3,Black,Grey,염료_total
1748,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
3730,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
308,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
930,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
49,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,6.0
1895,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5
1020,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
352,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.5


In [ ]:
#전체 변수 보기
np.array(data.columns)

array(['소재_경사정보_V1', '소재_경사정보_V2', '소재_경사정보_V3', '소재_경사정보_V4',
       '소재_경사정보_V5', '소재_경사정보_V6', '소재_경사정보_V7', '소재_경사정보_V8',
       '소재_경사정보_V9', '소재_경사정보_V10', '소재_경사정보_V11', '소재_경사정보_V12',
       '소재_경사정보_V13', '소재_경사정보_V14', '소재_경사정보_V15', '소재_경사정보_V16',
       '소재_위사정보_V1', '소재_위사정보_V2', '소재_위사정보_V3', '소재_위사정보_V4',
       '소재_위사정보_V5', '소재_위사정보_V6', '소재_위사정보_V7', '소재_위사정보_V8',
       '소재_위사정보_V9', '소재_위사정보_V10', '소재_위사정보_V11', '소재_위사정보_V12',
       '소재_위사정보_V13', '소재_위사정보_V14', '소재_위사정보_V15', '소재_위사정보_V16',
       '소재_제직정보_V1', '소재_제직정보_V2', '소재_제직정보_V3', '소재_제직정보_V4',
       '전처리_정보_V1', '전처리_정보_V2', '전처리_정보_V3', '전처리_정보_V4', '전처리_정보_V5',
       '전처리_정보_V6', '전처리_정보_V7', '전처리_정보_V8', '전처리_정보_V9', '전처리_정보_V10',
       '전처리_정보_V11', '전처리_정보_V12', '전처리_정보_V13', '전처리_정보_V14',
       '전처리_정보_V15', '전처리_정보_V16', '전처리_정보_V17', '전처리_정보_V18',
       '전처리_정보_V19', '전처리_정보_V20', '전처리_정보_V21', '전처리_정보_V22',
       '전처리_정보_V23', '전처리_정보_V24', '전처리_정보_V25', '전처리_정보_V26',
       '전처리_정보_V27', '

In [ ]:
len(np.array(data.columns))

165

# 범주형 및 수치형 인코딩

In [ ]:
# #수치형 변수 담기
# num_feature_col=[ '소재_경사정보_V2',
#  '소재_경사정보_V3',
#  '소재_경사정보_V6',
#  '소재_경사정보_V8',
#  '소재_경사정보_V9',
#  '소재_경사정보_V10',
#  '소재_경사정보_V11',
#  '소재_경사정보_V12',
#  '소재_경사정보_V13',
#  '소재_경사정보_V14',
#  '소재_경사정보_V15',
#  '소재_위사정보_V2',
#  '소재_위사정보_V3',
#  '소재_위사정보_V6',
#  '소재_위사정보_V8',
#  '소재_위사정보_V9',
#  '소재_위사정보_V10',
#  '소재_위사정보_V11',
#  '소재_위사정보_V12',
#  '소재_위사정보_V13',
#  '소재_위사정보_V14',
#  '소재_위사정보_V15',
#  '소재_제직정보_V3',
#  '소재_제직정보_V4',
#  '전처리_정보_V1',
#  '전처리_정보_V2',
#  '전처리_정보_V3',
#  '전처리_정보_V4',
#  '전처리_정보_V5',
#  '전처리_정보_V6',
#  '전처리_정보_V7',
#  '전처리_정보_V8',
#  '전처리_정보_V9',
#  '전처리_정보_V10',
#  '전처리_정보_V11',
#  '전처리_정보_V12',
#  '전처리_정보_V13',
#  '전처리_정보_V14',
#  '전처리_정보_V15',
#  '전처리_정보_V16',
#  '전처리_정보_V17',
#  '전처리_정보_V18',
#  '전처리_정보_V19',
#  '전처리_정보_V20',
#  '전처리_정보_V21',
#  '전처리_정보_V22',
#  '전처리_정보_V23',
#  '전처리_정보_V24',
#  '전처리_정보_V25',
#  '전처리_정보_V26',
#  '전처리_정보_V27',
#  '전처리_정보_V28',
#  '전처리_정보_V29',
#  '전처리_정보_V30',
#  '전처리_정보_V31',
#  '전처리_정보_V32',
#  '전처리_정보_V33',
#  '전처리_정보_V34',
#  '배합_염료1_Yellow',
#  '배합_염료1_Red',
#  '배합_염료1_Blue',
#  '배합_염료2_Yellow',
#  '배합_염료2_Red',
#  '배합_염료2_Blue',
#  '배합_염료3_Yellow',
#  '배합_염료3_Red',
#  '배합_염료3_Blue',
#  '배합_염료1_Black',
#  '배합_염료2_Black',
#  '배합_염료3_Black',
#  '배합_염료1_Grey',
#  '배합_염료2_Grey',
#  '배합_염료3_Grey',
#  '배합_염료2_Brown',
#  '배합_염료1_Brown',
#  '배합_염료3_Brown',
#  '배합_조제_V1',
#  '배합_조제_V2',
#  '배합_조제_V3',
#  '배합_조제_V4',
#  '배합_조제_V5',
#  '배합_조제_V6',
#  '염색_정보_V1',
#  '염색_정보_V2',
#  '염색_정보_V3',
#  '염색_정보_V4',
#  '염색_정보_V5',
#  '염색_정보_V6',
#  '염색_정보_V7',
#  '염색_정보_V8',
#  '염색_정보_V9',
#  '염색_정보_V10',
#  '염색_정보_V11',
#  '염색_정보_V12',
#  '염색_정보_V13',
#  '염색_정보_V14',
#  '염색_정보_V15',
#  '염색_정보_V16',
#  '염색_정보_V17',
#  '염색_정보_V18',
#  '염색_정보_V19',
#  '염색_정보_V20',
#  '염색_정보_V21',
#  '염색_정보_V22',
#  '후처리_정보_V1',
#  '후처리_정보_V2',
#  '후처리_정보_V3',
#  '후처리_정보_V4',
#  '후처리_정보_V5',
#  '후처리_정보_V6',
#  '후처리_정보_V7',
#  '후처리_정보_V8',
#  '후처리_정보_V9',
#  '후처리_정보_V10',
#  '후처리_정보_V11',
#  '후처리_정보_V12',
#  '후처리_정보_V13',
#  '후처리_정보_V14',
#  '후처리_정보_V15',
#  '후처리_정보_V16',
#  '후처리_정보_V17',
#  '후처리_정보_V18',
#  '후처리_정보_V19',
#  '후처리_정보_V20',
#  '후처리_정보_V21',
#  '후처리_정보_V22',
#  '후처리_정보_V23',
#  '후처리_정보_V24',
#  '후처리_정보_V25',
#  '후처리_정보_V26',
#  '후처리_정보_V27',
#  '후처리_정보_V28',
#  '후처리_정보_V29',
#  '후처리_정보_V30',
#  '후가공_정보_V1',
#  '후가공_정보_V2',
#  '후가공_정보_V3',
#  '후가공_정보_V4',
#  '후가공_정보_V5',
#  '후가공_정보_V6',
#  '배합_염료1_Yellow_prop',
#  '배합_염료1_Red_prop',
#  '배합_염료1_Blue_prop',
#  '배합_염료2_Yellow_prop',
#  '배합_염료2_Red_prop',
#  '배합_염료2_Blue_prop',
#  '배합_염료3_Yellow_prop',
#  '배합_염료3_Red_prop',
#  '배합_염료3_Blue_prop',
#  '배합_염료1_Black_prop',
#  '배합_염료2_Black_prop',
#  '배합_염료3_Black_prop',
#  '배합_염료1_Grey_prop',
#  '배합_염료2_Grey_prop',
#  '배합_염료3_Grey_prop',
#  '배합_염료2_Brown_prop',
#  '배합_염료1_Brown_prop',
#  '배합_염료3_Brown_prop',
#  '색상_L*',
#  '색상_a*',
#  '색상_b*'
#                  ]

# #범주형 변수 담기
# cate_feature_col=['소재_경사정보_V1',
#  '소재_경사정보_V4',
#  '소재_경사정보_V5',
#  '소재_경사정보_V7',
#  '소재_경사정보_V16',
#  '소재_위사정보_V1',
#  '소재_위사정보_V4',
#  '소재_위사정보_V5',
#  '소재_위사정보_V7',
#  '소재_위사정보_V16',
#  '소재_제직정보_V1',
#  '소재_제직정보_V2'
#                  ]

In [ ]:
#수치형 변수 담기
num_feature_col=[ '소재_경사정보_V2',
 '소재_경사정보_V3',
 '소재_경사정보_V6',
 '소재_경사정보_V8',
 '소재_경사정보_V9',
 '소재_경사정보_V10',
 '소재_경사정보_V11',
 '소재_경사정보_V12',
 '소재_경사정보_V13',
 '소재_경사정보_V14',
 '소재_경사정보_V15',
 '소재_위사정보_V2',
 '소재_위사정보_V3',
 '소재_위사정보_V6',
 '소재_위사정보_V8',
 '소재_위사정보_V9',
 '소재_위사정보_V10',
 '소재_위사정보_V11',
 '소재_위사정보_V12',
 '소재_위사정보_V13',
 '소재_위사정보_V14',
 '소재_위사정보_V15',
 '소재_제직정보_V3',
 '소재_제직정보_V4',
 '전처리_정보_V1',
 '전처리_정보_V2',
 '전처리_정보_V3',
 '전처리_정보_V4',
 '전처리_정보_V5',
 '전처리_정보_V6',
 '전처리_정보_V7',
 '전처리_정보_V8',
 '전처리_정보_V9',
 '전처리_정보_V10',
 '전처리_정보_V11',
 '전처리_정보_V12',
 '전처리_정보_V13',
 '전처리_정보_V14',
 '전처리_정보_V15',
 '전처리_정보_V16',
 '전처리_정보_V17',
 '전처리_정보_V18',
 '전처리_정보_V19',
 '전처리_정보_V20',
 '전처리_정보_V21',
 '전처리_정보_V22',
 '전처리_정보_V23',
 '전처리_정보_V24',
 '전처리_정보_V25',
 '전처리_정보_V26',
 '전처리_정보_V27',
 '전처리_정보_V28',
 '전처리_정보_V29',
 '전처리_정보_V30',
 '전처리_정보_V31',
 '전처리_정보_V32',
 '전처리_정보_V33',
 '전처리_정보_V34',
 '배합_조제_V1',
 '배합_조제_V2',
 '배합_조제_V3',
 '배합_조제_V4',
 '배합_조제_V5',
 '배합_조제_V6',
 '염색_정보_V1',
 '염색_정보_V2',
 '염색_정보_V3',
 '염색_정보_V4',
 '염색_정보_V5',
 '염색_정보_V6',
 '염색_정보_V7',
 '염색_정보_V8',
 '염색_정보_V9',
 '염색_정보_V10',
 '염색_정보_V11',
 '염색_정보_V12',
 '염색_정보_V13',
 '염색_정보_V14',
 '염색_정보_V15',
 '염색_정보_V16',
 '염색_정보_V17',
 '염색_정보_V18',
 '염색_정보_V19',
 '염색_정보_V20',
 '염색_정보_V21',
 '염색_정보_V22',
 '후처리_정보_V1',
 '후처리_정보_V2',
 '후처리_정보_V3',
 '후처리_정보_V4',
 '후처리_정보_V5',
 '후처리_정보_V6',
 '후처리_정보_V7',
 '후처리_정보_V8',
 '후처리_정보_V9',
 '후처리_정보_V10',
 '후처리_정보_V11',
 '후처리_정보_V12',
 '후처리_정보_V13',
 '후처리_정보_V14',
 '후처리_정보_V15',
 '후처리_정보_V16',
 '후처리_정보_V17',
 '후처리_정보_V18',
 '후처리_정보_V19',
 '후처리_정보_V20',
 '후처리_정보_V21',
 '후처리_정보_V22',
 '후처리_정보_V23',
 '후처리_정보_V24',
 '후처리_정보_V25',
 '후처리_정보_V26',
 '후처리_정보_V27',
 '후처리_정보_V28',
 '후처리_정보_V29',
 '후처리_정보_V30',
 '후가공_정보_V1',
 '후가공_정보_V2',
 '후가공_정보_V3',
 '후가공_정보_V4',
 '후가공_정보_V5',
 '후가공_정보_V6',
 '배합_염료1_Yellow_prop',
 '배합_염료1_Red_prop',
 '배합_염료1_Blue_prop',
 '배합_염료2_Yellow_prop',
 '배합_염료2_Red_prop',
 '배합_염료2_Blue_prop',
 '배합_염료3_Yellow_prop',
 '배합_염료3_Red_prop',
 '배합_염료3_Blue_prop',
 '배합_염료1_Black_prop',
 '배합_염료2_Black_prop',
 '배합_염료3_Black_prop',
 '배합_염료1_Grey_prop',
 '배합_염료2_Grey_prop',
 '배합_염료3_Grey_prop',
 '배합_염료2_Brown_prop',
 '배합_염료1_Brown_prop',
 '배합_염료3_Brown_prop',
            #  '배합_염료1_Yellow', '배합_염료1_Red', '배합_염료1_Blue', '배합_염료2_Yellow',
           # '배합_염료2_Red', '배합_염료2_Blue', '배합_염료3_Yellow', '배합_염료3_Red',
           # '배합_염료3_Blue', '배합_염료1_Black', '배합_염료2_Black', '배합_염료3_Black',
           # '배합_염료1_Grey', '배합_염료2_Grey', '배합_염료3_Grey', '배합_염료2_Brown',
           # '배합_염료1_Brown', '배합_염료3_Brown',\n",
           '염료1', 'Red', 'Blue', 'Yellow', '염료2', 'Brown', '염료3', 'Black', 'Grey',
           '염료_total',
 '색상_L*',
 '색상_a*',
 '색상_b*'
                 ]

#범주형 변수 담기
cate_feature_col=['소재_경사정보_V1',
 '소재_경사정보_V4',
 '소재_경사정보_V5',
 '소재_경사정보_V7',
 '소재_경사정보_V16',
 '소재_위사정보_V1',
 '소재_위사정보_V4',
 '소재_위사정보_V5',
 '소재_위사정보_V7',
 '소재_위사정보_V16',
 '소재_제직정보_V1',
 '소재_제직정보_V2'
                 ]

In [ ]:
#변수 갯수
len(num_feature_col)+len(cate_feature_col)

165

### 라벨인코더해서 카테고리퀄 데이터 수치화

In [ ]:
# Category feature 숫자로 인코딩
categorical_names = {}
for feature in cate_feature_col:
    le = LabelEncoder()
    le.fit(data[feature])
    data[feature+"_enc"] = le.transform(data[feature])
    categorical_names[feature] = le.classes_

In [ ]:
categorical_names

{'소재_경사정보_V1': array(['PET 재생 섬유'], dtype=object),
 '소재_경사정보_V4': array(['Semi Dull'], dtype=object),
 '소재_경사정보_V5': array(['Draw Textured Yarn'], dtype=object),
 '소재_경사정보_V7': array(['Filaments'], dtype=object),
 '소재_경사정보_V16': array(['Interlace'], dtype=object),
 '소재_위사정보_V1': array(['PET 재생 섬유'], dtype=object),
 '소재_위사정보_V4': array(['Semi Dull'], dtype=object),
 '소재_위사정보_V5': array(['Draw Textured Yarn'], dtype=object),
 '소재_위사정보_V7': array(['Filaments'], dtype=object),
 '소재_위사정보_V16': array(['Interlace'], dtype=object),
 '소재_제직정보_V1': array(['WOVEN'], dtype=object),
 '소재_제직정보_V2': array(['TWILL'], dtype=object)}

In [ ]:
# #min max 정규화
# std_model = {}
# for feature in num_feature_col:
#     std = MinMaxScaler()
#     print(feature)
#     std.fit(data[[feature]])
#     data[feature+"_std"] = std.transform(data[[feature]])
#     std_model[feature] = std

In [ ]:
#min max 정규화
std_model = {}
for feature in num_feature_col:
    std = MinMaxScaler()
    print(feature)
    std.fit(data[[feature]])

    if feature.endswith("_prop"):
        data[feature+"_std"] = data[feature]
    else:
        data[feature+"_std"] = std.transform(data[[feature]])
    std_model[feature] = std

소재_경사정보_V2
소재_경사정보_V3
소재_경사정보_V6
소재_경사정보_V8
소재_경사정보_V9
소재_경사정보_V10
소재_경사정보_V11
소재_경사정보_V12
소재_경사정보_V13
소재_경사정보_V14
소재_경사정보_V15
소재_위사정보_V2
소재_위사정보_V3
소재_위사정보_V6
소재_위사정보_V8
소재_위사정보_V9
소재_위사정보_V10
소재_위사정보_V11
소재_위사정보_V12
소재_위사정보_V13
소재_위사정보_V14
소재_위사정보_V15
소재_제직정보_V3
소재_제직정보_V4
전처리_정보_V1
전처리_정보_V2
전처리_정보_V3
전처리_정보_V4
전처리_정보_V5
전처리_정보_V6
전처리_정보_V7
전처리_정보_V8
전처리_정보_V9
전처리_정보_V10
전처리_정보_V11
전처리_정보_V12
전처리_정보_V13
전처리_정보_V14
전처리_정보_V15
전처리_정보_V16
전처리_정보_V17
전처리_정보_V18
전처리_정보_V19
전처리_정보_V20
전처리_정보_V21
전처리_정보_V22
전처리_정보_V23
전처리_정보_V24
전처리_정보_V25
전처리_정보_V26
전처리_정보_V27
전처리_정보_V28
전처리_정보_V29
전처리_정보_V30
전처리_정보_V31
전처리_정보_V32
전처리_정보_V33
전처리_정보_V34
배합_조제_V1
배합_조제_V2
배합_조제_V3
배합_조제_V4
배합_조제_V5
배합_조제_V6
염색_정보_V1
염색_정보_V2
염색_정보_V3
염색_정보_V4
염색_정보_V5
염색_정보_V6
염색_정보_V7
염색_정보_V8
염색_정보_V9
염색_정보_V10
염색_정보_V11
염색_정보_V12
염색_정보_V13
염색_정보_V14
염색_정보_V15
염색_정보_V16
염색_정보_V17
염색_정보_V18
염색_정보_V19
염색_정보_V20
염색_정보_V21
염색_정보_V22
후처리_정보_V1
후처리_정보_V2
후처리_정보_V3
후처리_정보_V4
후처리_정보_V5
후처리_정보_V6
후처리_정보_V7
후처리_정보_V8
후처리_정보_V9
후처리_

In [ ]:
data[['색상_L*',
'색상_a*',
'색상_b*']]

,색상_L*,색상_a*,색상_b*
1748,20.7300,8.2000,-20.2100
3730,72.4890,25.1515,84.0940
308,50.1900,6.8500,-20.2500
930,50.6250,62.5350,17.1050
49,57.1100,34.0900,55.9900
...,...,...,...
1094,54.2400,9.0675,9.6225
1895,65.3550,43.9665,22.5465
1020,34.3850,-18.0250,1.6150
352,45.4500,-15.8800,25.3500


In [ ]:
np.array(data.columns)

array(['소재_경사정보_V1', '소재_경사정보_V2', '소재_경사정보_V3', '소재_경사정보_V4',
       '소재_경사정보_V5', '소재_경사정보_V6', '소재_경사정보_V7', '소재_경사정보_V8',
       '소재_경사정보_V9', '소재_경사정보_V10', '소재_경사정보_V11', '소재_경사정보_V12',
       '소재_경사정보_V13', '소재_경사정보_V14', '소재_경사정보_V15', '소재_경사정보_V16',
       '소재_위사정보_V1', '소재_위사정보_V2', '소재_위사정보_V3', '소재_위사정보_V4',
       '소재_위사정보_V5', '소재_위사정보_V6', '소재_위사정보_V7', '소재_위사정보_V8',
       '소재_위사정보_V9', '소재_위사정보_V10', '소재_위사정보_V11', '소재_위사정보_V12',
       '소재_위사정보_V13', '소재_위사정보_V14', '소재_위사정보_V15', '소재_위사정보_V16',
       '소재_제직정보_V1', '소재_제직정보_V2', '소재_제직정보_V3', '소재_제직정보_V4',
       '전처리_정보_V1', '전처리_정보_V2', '전처리_정보_V3', '전처리_정보_V4', '전처리_정보_V5',
       '전처리_정보_V6', '전처리_정보_V7', '전처리_정보_V8', '전처리_정보_V9', '전처리_정보_V10',
       '전처리_정보_V11', '전처리_정보_V12', '전처리_정보_V13', '전처리_정보_V14',
       '전처리_정보_V15', '전처리_정보_V16', '전처리_정보_V17', '전처리_정보_V18',
       '전처리_정보_V19', '전처리_정보_V20', '전처리_정보_V21', '전처리_정보_V22',
       '전처리_정보_V23', '전처리_정보_V24', '전처리_정보_V25', '전처리_정보_V26',
       '전처리_정보_V27', '

In [ ]:
#독립변수 X에 담기
xxx1=np.where(data.columns=='소재_경사정보_V1_enc')[0][0]
xxx2=np.where(data.columns=='색상_L*_std')[0][0]
X=data[data.columns[xxx1:(xxx2)]]
#X=pd.concat([X,data_빅웨이브[['cor0_enc']]],axis=1)
X.index=data.index
X.to_csv("/content/da/t_s.csv")

In [ ]:
#종속변수 Y에 담기
yyy1=np.where(data.columns=='색상_L*')[0][0]
yyy2=np.where(data.columns=='색상_b*')[0][0]
Y=data[data.columns[yyy1:(yyy2+1)]]
Y.index=data.index
Y.to_csv("/content/da/te_s.csv")

In [ ]:
X[:,100:110]

InvalidIndexError: ignored

In [ ]:
data

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,Yellow_std,염료2_std,Brown_std,염료3_std,Black_std,Grey_std,염료_total_std,색상_L*_std,색상_a*_std,색상_b*_std
1748,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.056672,0.329261,0.093260
3730,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.495798,0.750613,0.533710,0.962131
308,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.075630,0.451647,0.312979,0.092927
930,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.327731,0.457479,0.984586,0.404101
49,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.411765,0.544425,0.641516,0.728020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.000000,0.505946,0.339724,0.341770
1895,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.075630,0.654967,0.760635,0.449430
1020,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.495798,0.239747,0.012965,0.275067
352,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.243697,0.388097,0.038836,0.472783


# LSTM 모델 설계

In [ ]:
#카테고리변수 범주확인
for i in cate_feature_col:
    print(i,data[i].nunique())

소재_경사정보_V1 1
소재_경사정보_V4 1
소재_경사정보_V5 1
소재_경사정보_V7 1
소재_경사정보_V16 1
소재_위사정보_V1 1
소재_위사정보_V4 1
소재_위사정보_V5 1
소재_위사정보_V7 1
소재_위사정보_V16 1
소재_제직정보_V1 1
소재_제직정보_V2 1


In [ ]:
#수치형 변수 확인
print(len(num_feature_col))

num_feature_col

153


['소재_경사정보_V2',
 '소재_경사정보_V3',
 '소재_경사정보_V6',
 '소재_경사정보_V8',
 '소재_경사정보_V9',
 '소재_경사정보_V10',
 '소재_경사정보_V11',
 '소재_경사정보_V12',
 '소재_경사정보_V13',
 '소재_경사정보_V14',
 '소재_경사정보_V15',
 '소재_위사정보_V2',
 '소재_위사정보_V3',
 '소재_위사정보_V6',
 '소재_위사정보_V8',
 '소재_위사정보_V9',
 '소재_위사정보_V10',
 '소재_위사정보_V11',
 '소재_위사정보_V12',
 '소재_위사정보_V13',
 '소재_위사정보_V14',
 '소재_위사정보_V15',
 '소재_제직정보_V3',
 '소재_제직정보_V4',
 '전처리_정보_V1',
 '전처리_정보_V2',
 '전처리_정보_V3',
 '전처리_정보_V4',
 '전처리_정보_V5',
 '전처리_정보_V6',
 '전처리_정보_V7',
 '전처리_정보_V8',
 '전처리_정보_V9',
 '전처리_정보_V10',
 '전처리_정보_V11',
 '전처리_정보_V12',
 '전처리_정보_V13',
 '전처리_정보_V14',
 '전처리_정보_V15',
 '전처리_정보_V16',
 '전처리_정보_V17',
 '전처리_정보_V18',
 '전처리_정보_V19',
 '전처리_정보_V20',
 '전처리_정보_V21',
 '전처리_정보_V22',
 '전처리_정보_V23',
 '전처리_정보_V24',
 '전처리_정보_V25',
 '전처리_정보_V26',
 '전처리_정보_V27',
 '전처리_정보_V28',
 '전처리_정보_V29',
 '전처리_정보_V30',
 '전처리_정보_V31',
 '전처리_정보_V32',
 '전처리_정보_V33',
 '전처리_정보_V34',
 '배합_조제_V1',
 '배합_조제_V2',
 '배합_조제_V3',
 '배합_조제_V4',
 '배합_조제_V5',
 '배합_조제_V6',
 '염색_정보_V1',
 '염색_정보_V2',
 '염색_정보_V3',
 '염색_정보_V4

In [ ]:
#소재 변수들중 수치형 변수 확인
num_feature_col[0:24]

['소재_경사정보_V2',
 '소재_경사정보_V3',
 '소재_경사정보_V6',
 '소재_경사정보_V8',
 '소재_경사정보_V9',
 '소재_경사정보_V10',
 '소재_경사정보_V11',
 '소재_경사정보_V12',
 '소재_경사정보_V13',
 '소재_경사정보_V14',
 '소재_경사정보_V15',
 '소재_위사정보_V2',
 '소재_위사정보_V3',
 '소재_위사정보_V6',
 '소재_위사정보_V8',
 '소재_위사정보_V9',
 '소재_위사정보_V10',
 '소재_위사정보_V11',
 '소재_위사정보_V12',
 '소재_위사정보_V13',
 '소재_위사정보_V14',
 '소재_위사정보_V15',
 '소재_제직정보_V3',
 '소재_제직정보_V4']

In [ ]:
#전처리 사용관련 수치형 변수 확인

num_feature_col[24:52]
#print(len(num_feature_col[14:44]))

['전처리_정보_V1',
 '전처리_정보_V2',
 '전처리_정보_V3',
 '전처리_정보_V4',
 '전처리_정보_V5',
 '전처리_정보_V6',
 '전처리_정보_V7',
 '전처리_정보_V8',
 '전처리_정보_V9',
 '전처리_정보_V10',
 '전처리_정보_V11',
 '전처리_정보_V12',
 '전처리_정보_V13',
 '전처리_정보_V14',
 '전처리_정보_V15',
 '전처리_정보_V16',
 '전처리_정보_V17',
 '전처리_정보_V18',
 '전처리_정보_V19',
 '전처리_정보_V20',
 '전처리_정보_V21',
 '전처리_정보_V22',
 '전처리_정보_V23',
 '전처리_정보_V24',
 '전처리_정보_V25',
 '전처리_정보_V26',
 '전처리_정보_V27',
 '전처리_정보_V28']

In [ ]:
#전처리 다음과정 사용관련 수치형 변수 확인
num_feature_col[52:58]

['전처리_정보_V29',
 '전처리_정보_V30',
 '전처리_정보_V31',
 '전처리_정보_V32',
 '전처리_정보_V33',
 '전처리_정보_V34']

In [ ]:
#배합 변수들중 수치형 변수 확인
num_feature_col[58:82]

['배합_조제_V1',
 '배합_조제_V2',
 '배합_조제_V3',
 '배합_조제_V4',
 '배합_조제_V5',
 '배합_조제_V6',
 '염색_정보_V1',
 '염색_정보_V2',
 '염색_정보_V3',
 '염색_정보_V4',
 '염색_정보_V5',
 '염색_정보_V6',
 '염색_정보_V7',
 '염색_정보_V8',
 '염색_정보_V9',
 '염색_정보_V10',
 '염색_정보_V11',
 '염색_정보_V12',
 '염색_정보_V13',
 '염색_정보_V14',
 '염색_정보_V15',
 '염색_정보_V16',
 '염색_정보_V17',
 '염색_정보_V18']

In [ ]:
#염색 변수들 수치형 변수 확인
num_feature_col[82:104]

['염색_정보_V19',
 '염색_정보_V20',
 '염색_정보_V21',
 '염색_정보_V22',
 '후처리_정보_V1',
 '후처리_정보_V2',
 '후처리_정보_V3',
 '후처리_정보_V4',
 '후처리_정보_V5',
 '후처리_정보_V6',
 '후처리_정보_V7',
 '후처리_정보_V8',
 '후처리_정보_V9',
 '후처리_정보_V10',
 '후처리_정보_V11',
 '후처리_정보_V12',
 '후처리_정보_V13',
 '후처리_정보_V14',
 '후처리_정보_V15',
 '후처리_정보_V16',
 '후처리_정보_V17',
 '후처리_정보_V18']

In [ ]:
#후처리 변수들중 수치형 변수 확인
num_feature_col[104:134]

['후처리_정보_V19',
 '후처리_정보_V20',
 '후처리_정보_V21',
 '후처리_정보_V22',
 '후처리_정보_V23',
 '후처리_정보_V24',
 '후처리_정보_V25',
 '후처리_정보_V26',
 '후처리_정보_V27',
 '후처리_정보_V28',
 '후처리_정보_V29',
 '후처리_정보_V30',
 '후가공_정보_V1',
 '후가공_정보_V2',
 '후가공_정보_V3',
 '후가공_정보_V4',
 '후가공_정보_V5',
 '후가공_정보_V6',
 '배합_염료1_Yellow_prop',
 '배합_염료1_Red_prop',
 '배합_염료1_Blue_prop',
 '배합_염료2_Yellow_prop',
 '배합_염료2_Red_prop',
 '배합_염료2_Blue_prop',
 '배합_염료3_Yellow_prop',
 '배합_염료3_Red_prop',
 '배합_염료3_Blue_prop',
 '배합_염료1_Black_prop',
 '배합_염료2_Black_prop',
 '배합_염료3_Black_prop']

In [ ]:
#후가공 변수들중 패더쪽 수치형 변수 확인
num_feature_col[134:136]

['배합_염료1_Grey_prop', '배합_염료2_Grey_prop']

In [ ]:
#후가공 변수들중 텐터쪽 수치형 변수 확인
num_feature_col[136:140]

['배합_염료3_Grey_prop',
 '배합_염료2_Brown_prop',
 '배합_염료1_Brown_prop',
 '배합_염료3_Brown_prop']

In [ ]:
#배합변수들중 파생변수로 생성된 염료 비율 변수들 확인
num_feature_col[140:158]

['염료1',
 'Red',
 'Blue',
 'Yellow',
 '염료2',
 'Brown',
 '염료3',
 'Black',
 'Grey',
 '염료_total',
 '색상_L*',
 '색상_a*',
 '색상_b*']

In [ ]:
#종속변수 확인
num_feature_col[158:161]

[]

In [ ]:
for i in cate_feature_col:
    print(i,data[i].nunique())

소재_경사정보_V1 1
소재_경사정보_V4 1
소재_경사정보_V5 1
소재_경사정보_V7 1
소재_경사정보_V16 1
소재_위사정보_V1 1
소재_위사정보_V4 1
소재_위사정보_V5 1
소재_위사정보_V7 1
소재_위사정보_V16 1
소재_제직정보_V1 1
소재_제직정보_V2 1


In [ ]:
len(cate_feature_col)

12

In [ ]:
len(num_feature_col)


153

In [ ]:
num_input = keras.layers.Input((len(num_feature_col)-3,))
num_input

<KerasTensor: shape=(None, 150) dtype=float32 (created by layer 'input_1')>

In [ ]:
X_train_df=X.loc[train_index]
X_test_df=X.loc[test_index]
y_train_df=Y.loc[train_index]
y_test_df=Y.loc[test_index]

In [ ]:
X_test_df

,소재_경사정보_V1_enc,소재_경사정보_V4_enc,소재_경사정보_V5_enc,소재_경사정보_V7_enc,소재_경사정보_V16_enc,소재_위사정보_V1_enc,소재_위사정보_V4_enc,소재_위사정보_V5_enc,소재_위사정보_V7_enc,소재_위사정보_V16_enc,...,염료1_std,Red_std,Blue_std,Yellow_std,염료2_std,Brown_std,염료3_std,Black_std,Grey_std,염료_total_std
3719,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.243697
2770,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.663866
3306,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159664
1674,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327731
2383,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.000000
1895,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.075630
1020,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.495798
352,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.243697


In [ ]:
# 모델입력 형태
# Category, Numeric, Enc(dummy) 나눠서 입력
X_train_cate_input = X_train_df[[j+"_enc" for j in cate_feature_col]].values
X_train_num_input = X_train_df[[j+"_std" for j in num_feature_col[:-3]]].values
Y_train_input = y_train_df.values

X_test_cate_input = X_test_df[[j+"_enc" for j in cate_feature_col]].values
X_test_num_input = X_test_df[[j+"_std" for j in num_feature_col[:-3]]].values
Y_test_input = y_test_df.values


X_test_cate_input = X_test_df[[j+"_enc" for j in cate_feature_col]].values
#X_test_cate_input = X_test_cate_input.reshape(1,X_test_cate_input.shape[0])
X_test_num_input = X_test_df[[j+"_std" for j in num_feature_col[:-3]]].values
#X_test_num_input = X_test_num_input.reshape(1,X_test_num_input.shape[0])

Y_test_input = y_test_df.values
#    Y_test_input = Y_test_input.reshape(1,Y_test_input.shape[0])


train_data2 = tf.data.Dataset.from_tensor_slices(((X_train_cate_input,X_train_num_input),
                                                Y_train_input)).batch(32).shuffle(len(X_train_cate_input))
#.shuffle(len(X_train_cate_input))

test_data2 = tf.data.Dataset.from_tensor_slices(((X_test_cate_input,X_test_num_input),
                                                Y_test_input)).batch(32)

In [ ]:
train_data2

<_ShuffleDataset element_spec=((TensorSpec(shape=(None, 12), dtype=tf.int64, name=None), TensorSpec(shape=(None, 150), dtype=tf.float64, name=None)), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>

In [ ]:
import torch
def rmse_loss(y_pred, y_true):
    mse_loss = torch.nn.MSELoss()
    rmse = torch.sqrt(mse_loss(y_pred, y_true))
    return rmse


In [ ]:
X

,소재_경사정보_V1_enc,소재_경사정보_V4_enc,소재_경사정보_V5_enc,소재_경사정보_V7_enc,소재_경사정보_V16_enc,소재_위사정보_V1_enc,소재_위사정보_V4_enc,소재_위사정보_V5_enc,소재_위사정보_V7_enc,소재_위사정보_V16_enc,...,염료1_std,Red_std,Blue_std,Yellow_std,염료2_std,Brown_std,염료3_std,Black_std,Grey_std,염료_total_std
1748,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
3730,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.495798
308,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075630
930,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327731
49,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.411765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.000000
1895,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.075630
1020,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.495798
352,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.243697


In [ ]:
# from pytorch_tabnet.tab_model import TabNetRegressor
# import numpy as np



# # PyTorch TabNetRegressor 모델 초기화
# tabnet_regressor = TabNetRegressor(optimizer_params=dict(lr=0.001), seed=42)

# # 데이터를 NumPy 배열로 변환 (이미 배열이라면 이 단계는 생략)
# X_train = np.concatenate((X_train_cate_input, X_train_num_input), axis=1)
# y_train = Y_train_input
# X_test = np.concatenate((X_test_cate_input, X_test_num_input), axis=1)
# y_test = Y_test_input

# # 모델 학습 (PyTorch TabNet 사용 가정)
# tabnet_regressor.fit(
#     X_train=X_train, y_train=y_train,
#     eval_set=[(X_train, y_train), (X_test, y_test)],
#     eval_name=['train', 'valid'],
#     eval_metric=['rmse'],  # 기본 제공되는 평가 지표 사용
#     max_epochs=1200,
#     loss_fn=rmse_loss,
#     batch_size=64,  # 배치 크기 조정
#     virtual_batch_size=128,
#     num_workers=0,
#     drop_last=False,
#     patience=50
# )

# # 학습 후 별도의 평가
# # 예측 수행
# predictions = tabnet_regressor.predict(X_test)

# # 사용자 정의 평가 지표를 사용하여 평가
# cie94_scores = cie94_color_difference(predictions, y_test)
# average_cie94 = np.mean(cie94_scores)
# print(f"Average CIE94 Color Difference: {average_cie94}")
# # 모델 학습 이후

# feature_importances_ = tabnet_regressor.feature_importances_
# print(feature_importances_)


In [ ]:
!pip install optuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.7 MB/s eta 0:00:00


In [ ]:
import optuna
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 여기에 필요한 데이터 로딩 및 전처리 코드 추가
# 예: X_train, y_train, X_test, y_test, X_train_cate_input, X_train_num_input 등

def objective(trial):
    # Optuna를 사용한 하이퍼파라미터 제안
    lr = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
    max_epochs = trial.suggest_int('max_epochs', 100, 2000)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    virtual_batch_size = trial.suggest_categorical('virtual_batch_size', [16, 32, 64, 128])
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)

    # 모델 초기화
    tabnet_regressor = TabNetRegressor(
        optimizer_params=dict(lr=lr),
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        seed=42
    )

    # 모델 학습
    tabnet_regressor.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_name=['train', 'valid'],
        eval_metric=['rmse'],
        loss_fn=rmse_loss,
        max_epochs=max_epochs,
        batch_size=batch_size,
        virtual_batch_size=virtual_batch_size,
        num_workers=0,
        drop_last=False,
        patience=50,
        # ... 기타 파라미터 ...
    )

    # 검증 세트에서의 성능 평가
    preds = tabnet_regressor.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 출력
print(f"Best RMSE: {study.best_value}")
print(f"Best hyperparameters: {study.best_params}")

# 최적의 하이퍼파라미터로 모델을 다시 학습
best_params = study.best_params
tabnet_regressor = TabNetRegressor(
    optimizer_params=dict(lr=best_params['lr']),
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    seed=42,
    max_epochs=best_params['max_epochs'],
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['virtual_batch_size']
)

tabnet_regressor.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['rmse'],
    loss_fn=rmse_loss,
    num_workers=0,
    drop_last=False,
    patience=50,
    # ... 기타 파라미터 ...
)

# 최적의 하이퍼파라미터를 사용한 모델로 예측 및 사용자 정의 평가 지표 계산
predictions = tabnet_regressor.predict(X_test)


# CIE94 계산 (cie94_color_difference 함수를 정의해야 함)
cie94_scores = cie94_color_difference(predictions, y_test)
average_cie94 = np.mean(cie94_scores)
print(f"Average CIE94 Color Difference with best hyperparameters: {average_cie94}")


[I 2023-11-24 08:56:50,703] A new study created in memory with name: no-name-d6696bed-42eb-4bfa-9d72-47f5e1d67161


epoch 0  | loss: 34.83069| train_rmse: 35.56223| valid_rmse: 34.83498|  0:00:08s
epoch 1  | loss: 28.18265| train_rmse: 30.06498| valid_rmse: 29.42152|  0:00:16s
epoch 2  | loss: 24.72065| train_rmse: 25.9685 | valid_rmse: 25.3778 |  0:00:18s
epoch 3  | loss: 23.64409| train_rmse: 23.93144| valid_rmse: 23.54471|  0:00:21s
epoch 4  | loss: 22.85687| train_rmse: 22.27755| valid_rmse: 21.92734|  0:00:25s
epoch 5  | loss: 21.93519| train_rmse: 21.2727 | valid_rmse: 20.9465 |  0:00:29s
epoch 6  | loss: 21.26366| train_rmse: 20.4515 | valid_rmse: 20.37916|  0:00:32s
epoch 7  | loss: 20.84698| train_rmse: 20.27563| valid_rmse: 20.13377|  0:00:35s
epoch 8  | loss: 20.35587| train_rmse: 19.24683| valid_rmse: 19.03977|  0:00:38s
epoch 9  | loss: 20.01037| train_rmse: 18.51022| valid_rmse: 18.30107|  0:00:42s
epoch 10 | loss: 19.424  | train_rmse: 18.45759| valid_rmse: 17.97607|  0:00:46s
epoch 11 | loss: 18.67119| train_rmse: 17.12233| valid_rmse: 16.94901|  0:00:49s
epoch 12 | loss: 17.94103| t

[I 2023-11-24 09:11:44,548] Trial 0 finished with value: 2.9935557003262057 and parameters: {'lr': 0.006962441123890509, 'max_epochs': 1258, 'batch_size': 128, 'virtual_batch_size': 16, 'n_d': 33, 'n_a': 18, 'n_steps': 7, 'gamma': 1.1288872968121477}. Best is trial 0 with value: 2.9935557003262057.


epoch 0  | loss: 39.07332| train_rmse: 38.83146| valid_rmse: 38.06097|  0:00:02s
epoch 1  | loss: 38.16164| train_rmse: 38.38513| valid_rmse: 37.61677|  0:00:04s
epoch 2  | loss: 37.03994| train_rmse: 37.51332| valid_rmse: 36.73511|  0:00:06s
epoch 3  | loss: 35.41031| train_rmse: 34.79863| valid_rmse: 34.2183 |  0:00:09s
epoch 4  | loss: 33.54884| train_rmse: 32.82616| valid_rmse: 32.28117|  0:00:12s
epoch 5  | loss: 31.25085| train_rmse: 30.01981| valid_rmse: 29.51984|  0:00:14s
epoch 6  | loss: 28.90279| train_rmse: 27.4004 | valid_rmse: 26.85216|  0:00:17s
epoch 7  | loss: 26.29614| train_rmse: 24.66188| valid_rmse: 24.28007|  0:00:19s
epoch 8  | loss: 22.80347| train_rmse: 21.04416| valid_rmse: 20.86608|  0:00:21s
epoch 9  | loss: 20.28103| train_rmse: 19.5084 | valid_rmse: 18.92124|  0:00:24s
epoch 10 | loss: 18.62708| train_rmse: 17.52642| valid_rmse: 17.33853|  0:00:27s
epoch 11 | loss: 17.40129| train_rmse: 16.72675| valid_rmse: 16.57974|  0:00:29s
epoch 12 | loss: 16.28108| t

[I 2023-11-24 09:21:10,598] Trial 1 finished with value: 2.4630146437592417 and parameters: {'lr': 0.0010270842676664053, 'max_epochs': 1009, 'batch_size': 64, 'virtual_batch_size': 64, 'n_d': 38, 'n_a': 35, 'n_steps': 4, 'gamma': 1.5163944899807087}. Best is trial 1 with value: 2.4630146437592417.


...

[I 2023-11-25 01:33:26,063] Trial 98 finished with value: 4.034034470002381 and parameters: {'lr': 0.02215289917571507, 'max_epochs': 1609, 'batch_size': 16, 'virtual_batch_size': 128, 'n_d': 45, 'n_a': 29, 'n_steps': 5, 'gamma': 1.3964497646428273}. Best is trial 71 with value: 1.310434919010017.


epoch 0  | loss: 32.82007| train_rmse: 28.85499| valid_rmse: 28.23301|  0:00:01s
epoch 1  | loss: 22.69547| train_rmse: 23.2558 | valid_rmse: 22.72259|  0:00:03s
epoch 2  | loss: 20.56892| train_rmse: 23.34556| valid_rmse: 22.83116|  0:00:04s
epoch 3  | loss: 20.32676| train_rmse: 21.65317| valid_rmse: 21.15793|  0:00:06s
epoch 4  | loss: 18.86779| train_rmse: 19.8604 | valid_rmse: 19.52196|  0:00:07s
epoch 5  | loss: 16.87627| train_rmse: 16.71208| valid_rmse: 16.84717|  0:00:09s
epoch 6  | loss: 14.7284 | train_rmse: 14.50175| valid_rmse: 14.58524|  0:00:10s
epoch 7  | loss: 12.80282| train_rmse: 13.11113| valid_rmse: 12.891  |  0:00:12s
epoch 8  | loss: 11.80949| train_rmse: 14.068  | valid_rmse: 13.39705|  0:00:14s
epoch 9  | loss: 10.36939| train_rmse: 10.85785| valid_rmse: 10.92189|  0:00:16s
epoch 10 | loss: 9.33699 | train_rmse: 13.51015| valid_rmse: 12.98751|  0:00:18s
epoch 11 | loss: 8.63231 | train_rmse: 9.7925  | valid_rmse: 9.04103 |  0:00:19s
epoch 12 | loss: 8.01551 | t

[I 2023-11-25 01:41:51,715] Trial 99 finished with value: 1.57308299503722 and parameters: {'lr': 0.014146289352032574, 'max_epochs': 1302, 'batch_size': 128, 'virtual_batch_size': 128, 'n_d': 39, 'n_a': 26, 'n_steps': 6, 'gamma': 1.274128674162997}. Best is trial 71 with value: 1.310434919010017.


Best RMSE: 1.310434919010017
Best hyperparameters: {'lr': 0.016295310674719794, 'max_epochs': 1403, 'batch_size': 128, 'virtual_batch_size': 128, 'n_d': 42, 'n_a': 21, 'n_steps': 5, 'gamma': 1.432071121475246}


TypeError: ignored

In [ ]:
# 한글 폰트 설치
!apt-get install -y -qq fonts-nanum

# 폰트 캐시를 삭제합니다.
!rm -rf /root/.cache/matplotlib/

# 폰트를 로드합니다.
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 설치된 폰트에서 Nanum 폰트의 전체 경로를 찾습니다.
nanum_font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

# Nanum 폰트를 Matplotlib의 기본 폰트로 설정합니다.
fm.fontManager.addfont(nanum_font_path)
plt.rc('font', family='NanumBarunGothic')

# 한글이 제대로 출력되는지 확인합니다.
plt.title('한글 제목 테스트')
plt.plot([0, 1], [0, 1])
plt.show()


In [ ]:
import matplotlib.font_manager as fm
fonts = [font.name for font in fm.fontManager.ttflist if 'Nanum' in font.name]
print(fonts)


In [ ]:
# 원본 특성 이름을 처리된 형태로 만들어줍니다.
feature_names = [j+"_enc" for j in cate_feature_col] + [j+"_std" for j in num_feature_col[:-3]]

# 이제 TabNetRegressor 모델로부터 특성 중요도를 가져옵니다.
feature_importances_ = tabnet_regressor.feature_importances_

# 중요도 순으로 특성을 정렬합니다.
sorted_idx = np.argsort(feature_importances_)[::-1]

# 상위 20개의 특성 이름과 중요도를 가져옵니다.
top_n = 30
sorted_feature_names_top = [feature_names[i] for i in sorted_idx[:top_n]]
sorted_importances_top = feature_importances_[sorted_idx[:top_n]]

# 시각화
plt.figure(figsize=(10, 8))
plt.barh(range(top_n), sorted_importances_top, align='center')
plt.yticks(range(top_n), sorted_feature_names_top)
plt.gca().invert_yaxis()  # 중요도가 높은 특성을 위로 표시합니다.
plt.xlabel('Importance')
plt.title('Top 20 Feature Importances')
plt.show()


In [ ]:
# 독립변수 X의 컬럼 이름을 리스트로 가져옵니다.
feature_names = X.columns.tolist()

# TabNet 모델로부터 특성 중요도를 가져옵니다.
feature_importances_ = tabnet_regressor.feature_importances_

# 중요도 순으로 특성을 정렬합니다.
sorted_idx = np.argsort(feature_importances_)[::-1]

# 상위 20개의 특성 이름과 중요도를 가져옵니다.
top_n = 20
sorted_feature_names_top = [feature_names[i] for i in sorted_idx[:top_n]]
sorted_importances_top = feature_importances_[sorted_idx[:top_n]]

# 시각화
plt.figure(figsize=(10, 8))
plt.barh(range(top_n), sorted_importances_top, align='center')
plt.yticks(range(top_n), sorted_feature_names_top)
plt.gca().invert_yaxis()  # 중요도가 높은 특성을 위로 표시합니다.
plt.xlabel('Importance')
plt.title('Top 20 Feature Importances')
plt.show()


In [ ]:
X_train_df.iloc[:,0:162]

,소재_경사정보_V1_enc,소재_경사정보_V4_enc,소재_경사정보_V5_enc,소재_경사정보_V7_enc,소재_경사정보_V16_enc,소재_위사정보_V1_enc,소재_위사정보_V4_enc,소재_위사정보_V5_enc,소재_위사정보_V7_enc,소재_위사정보_V16_enc,...,염료1_std,Red_std,Blue_std,Yellow_std,염료2_std,Brown_std,염료3_std,Black_std,Grey_std,염료_total_std
1748,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
3730,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.495798
308,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075630
930,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327731
49,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.411765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008403
3286,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.411765
1669,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159664
2625,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.075630


In [ ]:
def cie94_eval(preds, dmatrix):
    labels = dmatrix.get_label()  # 실제 레이블 값을 얻습니다.
    preds = preds.reshape(-1, 3)  # 예측값을 적절한 형태로 재구성합니다 (예시는 3차원 컬러 데이터를 가정).
    labels = labels.reshape(-1, 3)  # 실제 레이블도 적절한 형태로 재구성합니다.

    # CIE94 색차를 계산합니다.
    scores = cie94_color_difference(preds, labels)

    # 평균 평가 점수를 반환합니다.
    return 'cie94', np.mean(scores)


In [ ]:
import xgboost as xgb
import numpy as np


# XGBoost 모델 초기화
xgb_regressor = xgb.XGBRegressor(
    learning_rate=0.001,
    n_estimators=1200,
    objective='reg:squarederror',  # MSE 기반의 손실 함수
    verbosity=1

)

# 데이터를 NumPy 배열로 변환
X_train = np.concatenate((X_train_cate_input, X_train_num_input), axis=1)
y_train = Y_train_input
X_test = np.concatenate((X_test_cate_input, X_test_num_input), axis=1)
y_test = Y_test_input

# XGBoost 모델 학습 부분
xgb_regressor.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric=cie94_eval,  # 수정된 사용자 정의 CIE94 평가 지표 사용
    early_stopping_rounds=50,
    verbose=True
)

# 학습 후 별도의 평가
predictions = xgb_regressor.predict(X_test)

# 사용자 정의 평가 지표를 사용하여 평가
cie94_scores = cie94_color_difference(predictions.reshape(-1, 3), y_test.reshape(-1, 3))
average_cie94 = np.mean(cie94_scores)
print(f"Average CIE94 Color Difference: {average_cie94}")


In [ ]:
!pip install pytorch_tabnet


In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

In [ ]:
X_train_df=X.loc[train_index]
X_test_df=X.loc[test_index]
y_train_df=Y.loc[train_index]
y_test_df=Y.loc[test_index]

In [ ]:
X_train_df

,소재_경사정보_V1_enc,소재_경사정보_V4_enc,소재_경사정보_V5_enc,소재_경사정보_V7_enc,소재_경사정보_V16_enc,소재_위사정보_V1_enc,소재_위사정보_V4_enc,소재_위사정보_V5_enc,소재_위사정보_V7_enc,소재_위사정보_V16_enc,...,염료1_std,Red_std,Blue_std,Yellow_std,염료2_std,Brown_std,염료3_std,Black_std,Grey_std,염료_total_std
1748,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
3730,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.495798
308,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075630
930,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327731
49,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.411765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008403
3286,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.411765
1669,0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159664
2625,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.075630


In [ ]:
# X_train_df = X_train_df.to_numpy()
# X_test_df = X_test_df.to_numpy()
# y_train_df = y_train_df.to_numpy()
# y_test_df = y_test_df.to_numpy()

In [ ]:
# DataFrame을 NumPy 배열로 변환
X_train = X_train_df.values
y_train = y_train_df.values
X_test = X_test_df.values
y_test = y_test_df.values




In [ ]:
y_train.shape

In [ ]:
# from pytorch_tabnet.tab_model import TabNetRegressor

# # TabNetRegressor 모델 초기화
# tabnet_regressor = TabNetRegressor()


# # 모델 학습
# tabnet_regressor.fit(
#     X_train, y_train,
#     eval_set=[(X_train, y_train)],
#     eval_name=['test'],
#     eval_metric=['rmse'],  # RMSE (Root Mean Square Error)를 평가 지표로 사용
#     max_epochs=1200,
#     batch_size=32,
#     virtual_batch_size=128,
#     num_workers=0,
#     drop_last=False
# )

# # 모델 평가
# predictions = tabnet_regressor.predict(X_test)


In [ ]:
#LSTM 모델 설계
def model_build():
    keras.backend.clear_session()

    ##범주형변수, 수치형 변수 갯수 만큼 설정
    cate_input = keras.layers.Input((12,))
    num_input = keras.layers.Input((158,))

    #카테고리변수설정(모두 소재)
    emb1 = keras.layers.Embedding(1,8)(cate_input[:,0])
    emb2 = keras.layers.Embedding(1,8)(cate_input[:,1])
    emb3 = keras.layers.Embedding(1,8)(cate_input[:,2])
    emb4 = keras.layers.Embedding(1,8)(cate_input[:,3])
    emb5 = keras.layers.Embedding(1,8)(cate_input[:,4])
    emb6 = keras.layers.Embedding(1,8)(cate_input[:,5])
    emb7 = keras.layers.Embedding(1,8)(cate_input[:,6])
    emb8 = keras.layers.Embedding(1,8)(cate_input[:,7])
    emb9 = keras.layers.Embedding(1,8)(cate_input[:,8])
    emb10 = keras.layers.Embedding(1,8)(cate_input[:,9])
    emb11 = keras.layers.Embedding(1,8)(cate_input[:,10])
    emb12 = keras.layers.Embedding(1,8)(cate_input[:,11])

    #소재 부
    part1_layer = keras.layers.Concatenate()([num_input[:,0:24],
                                              emb1,emb2,emb3,emb4,
                                              emb5,emb6,emb7,emb8,
                                              emb9,emb10,emb11,emb12
                                             ])
    part1_layer = keras.layers.Dense(96,activation="selu")(part1_layer)
    part1_layer = keras.layers.Dropout(0.3)(part1_layer)
    part1_layer = tf.expand_dims(part1_layer,axis=1)

    # 전처리_1차
    part2_layer = keras.layers.Dense(24,activation="selu")(num_input[:,24:52])
    part2_layer = keras.layers.Dense(96,activation="selu")(part2_layer)
    part2_layer = keras.layers.Dropout(0.3)(part2_layer)
    part2_layer = tf.expand_dims(part2_layer,axis=1)

    # 전처리_2차
    part3_layer = keras.layers.Dense(24,activation="selu")(num_input[:,52:58])
    part3_layer = keras.layers.Dense(96,activation="selu")(part3_layer)
    part3_layer = keras.layers.Dropout(0.3)(part3_layer)
    part3_layer = tf.expand_dims(part3_layer,axis=1)

    # 배합부
    part4_layer = keras.layers.Concatenate()([num_input[:,58:82],num_input[:,140:158]])
    part4_layer = keras.layers.Dense(96,activation="selu")(part4_layer)
    part4_layer = keras.layers.Dropout(0.3)(part4_layer)
    part4_layer = tf.expand_dims(part4_layer,axis=1)

    # 염색부
    part5_layer = keras.layers.Dense(24,activation="selu")(num_input[:,82:104])
    part5_layer = keras.layers.Dense(96,activation="selu")(part5_layer)
    part5_layer = keras.layers.Dropout(0.3)(part5_layer)
    part5_layer = tf.expand_dims(part5_layer,axis=1)

    #후처리부
    part6_layer = keras.layers.Dense(24,activation="selu")(num_input[:,104:134])
    part6_layer = keras.layers.Dense(96,activation="selu")(part6_layer)
    part6_layer = keras.layers.Dropout(0.3)(part6_layer)
    part6_layer = tf.expand_dims(part6_layer,axis=1)

    #후가공_1차 공정
    part7_layer = keras.layers.Dense(24,activation="selu")(num_input[:,134:136])
    part7_layer = keras.layers.Dense(96,activation="selu")(part7_layer)
    part7_layer = keras.layers.Dropout(0.3)(part7_layer)
    part7_layer = tf.expand_dims(part7_layer,axis=1)

    #후가공_2차 공정
    part8_layer = keras.layers.Dense(24,activation="selu")(num_input[:,136:140])
    part8_layer = keras.layers.Dense(96,activation="selu")(part8_layer)
    part8_layer = keras.layers.Dropout(0.3)(part8_layer)
    part8_layer = tf.expand_dims(part8_layer,axis=1)

    # Concatenate
    total_concat_be = keras.layers.Concatenate(axis=1)([part1_layer,
                                                        part2_layer,
                                                        part3_layer,
                                                        part4_layer,
                                                        part5_layer,
                                                        part6_layer,
                                                        part7_layer,
                                                        part8_layer])
    total_concat_be2  = AttentionLayer(96,96,2,0.01)(total_concat_be)

    # LSTM Layer
    total_concat = keras.layers.LSTM(96)(total_concat_be)

    # Dense Layer
    total_concat = keras.layers.Concatenate()([total_concat,num_input,total_concat_be2])
    total_concat = keras.layers.Dense(96,activation="selu")(total_concat)

    output = keras.layers.Dense(3,name="l_loss")(total_concat)

    # CIE94 Keras Custom Loss 함수 설계
    def cie94(y_true, y_pred):
        """Calculate color difference by using CIE94 formulae

        See http://en.wikipedia.org/wiki/Color_difference or
        http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

        cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
        >>> 58.0
        cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
        >>> 0.0
        """

        L1, a1, b1 = y_true[:,0],y_true[:,1],y_true[:,2]
        L2, a2, b2 = y_pred[:,0],y_pred[:,1],y_pred[:,2]


        C1 = K.sqrt(K.pow(a1,2) + K.pow(b1,2))
        C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
        delta_L = L1 - L2
        delta_C = C1 - C2
        delta_a = a1 - a2
        delta_b = b1 - b2
        delta_H_square = K.pow(delta_a,2) + K.pow(delta_b,2) - K.pow(delta_C,2)
        loss_a = (K.sqrt(K.pow(delta_L,2)
                + K.pow(delta_C,2) / K.pow(1.0 + 0.045 * C1,2)
                + delta_H_square / K.pow(1.0 + 0.015 * C1,2)))
        return loss_a

    model = keras.models.Model(inputs=[cate_input,num_input],outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01),loss=[cie94], metrics=[cie94])
    return(model)

In [ ]:
model = model_build()

In [ ]:
X_test_num_input.shape

In [ ]:
#혹시나 변수들중 NULL값 확인
for i in range(len(pd.DataFrame(X_train_num_input).columns)):
    print(sum(pd.DataFrame(X_train_num_input)[i].isnull()))

In [ ]:
# ModelCheckpoint 콜백 정의
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/1109/program/기초데이터/savemodel.h5'  # 모델을 저장할 파일 경로
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,  # 가장 좋은 모델만 저장
    monitor='val_cie94',  # 모니터링할 지표 (validation_data에서의 cie94 평가 지표)
    mode='min',  # 최솟값을 찾기 위해 모니터링
    verbose=1  # 저장할 때 로그 출력
)


In [ ]:
#1차 학습
a=model.fit(train_data2,
            epochs=1200,
            validation_data=test_data2,
            callbacks=[model_checkpoint_callback])


In [ ]:
#모델 불러오기
model.load_weights("/content/drive/MyDrive/Colab Notebooks/1109/program/기초데이터/savemodel.h5")

In [ ]:
#카테고리 데이터 내보내기
#with open("./model/categorical_names_new.pkl","wb") as f:
#    pickle.dump(categorical_names,f)

# MinMaxScaler model pkl 내보내기
#with open("./model/Dytec_std_model_new.pkl", "wb") as f:
#     pickle.dump(std_model,f)

In [ ]:
#예측 데이터 생성
Y_pred=pd.DataFrame(model.predict([X_train_cate_input,X_train_num_input]))
Y_pred.columns=['색상_L*', '색상_a*', '색상_b*']
Y_pred.index=X_train_df.index

#예측데이터 재할당
pred_TRANS = Y_pred

#실제 데이터 재규격화
Y_TRANS=y_train_df
Y_TRANS.columns=[['색상_L*', '색상_a*', '색상_b*']]

#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)

#이상치 확인
train_color_diff=diff_color.sort_values(0,ascending=False)
train_color_diff=train_color_diff.loc[train_color_diff[0]>=100]
train_color_diff_index=list(train_color_diff.index)
train_color_diff_index

In [ ]:
np.mean(diff_color)

In [ ]:
#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)
diff_color

In [ ]:
#lab값을 rgb로 변경
y_test_rgb=lab_to_rgb(Y_TRANS)
y_test_rgb

In [ ]:
#lab값을 rgb로 변경
pred_rgb=lab_to_rgb(pred_TRANS)
pred_rgb

# Test 데이터 확인

In [ ]:
#예측 확인
model.predict([X_test_cate_input,X_test_num_input])[:5]

#예측 데이터 생성
Y_pred=pd.DataFrame(model.predict([X_test_cate_input,X_test_num_input]))
Y_pred.columns=['색상_L*', '색상_a*', '색상_b*']
Y_pred.index=X_test_df.index

#예측데이터 재할당
pred_TRANS = Y_pred
pred_TRANS

#실제 데이터 재규격화
Y_TRANS=y_test_df
Y_TRANS.columns=[['색상_L*', '색상_a*', '색상_b*']]
Y_TRANS

#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)
diff_color

#이상치 확인
test_color_diff=diff_color.sort_values(0,ascending=False)
test_color_diff= test_color_diff.loc[test_color_diff[0]>=100]
test_color_diff_index=list(test_color_diff.index)
test_color_diff_index


In [ ]:
Y_pred

In [ ]:
y_test_df

In [ ]:
#L*
r2_score(y_test_df[y_test_df.columns[0]],Y_pred[Y_pred.columns[0]])


In [ ]:
#a*
r2_score(y_test_df[y_test_df.columns[1]],Y_pred[Y_pred.columns[1]])


In [ ]:
#b*
r2_score(y_test_df[y_test_df.columns[2]],Y_pred[Y_pred.columns[2]])


In [ ]:
np.mean(diff_color)

In [ ]:
#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)
diff_color

In [ ]:
#lab값을 rgb로 변경
y_test_rgb=lab_to_rgb(Y_TRANS)
y_test_rgb

In [ ]:
#lab값을 rgb로 변경
pred_rgb=lab_to_rgb(pred_TRANS)
pred_rgb

# 그래프 그리기

In [ ]:
for i in range(len(y_test_rgb)//3+1):
    plt.figure(figsize=(15,10))
    plt.subplot(181)
    plt.imshow(Image.new("RGB", (50,50),(int(y_test_rgb['r'][y_test_rgb.index[3*i]]*255),int(y_test_rgb['g'][y_test_rgb.index[3*i]]*255),int(y_test_rgb['b'][y_test_rgb.index[3*i]]*255))))
    plt.title('index: '+str(y_test_rgb.index[3*i])+'\nReal Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(182)
    plt.imshow(Image.new("RGB", (50,50),(int(pred_rgb['r'][y_test_rgb.index[3*i]]*255),int(pred_rgb['g'][y_test_rgb.index[3*i]]*255),int(pred_rgb['b'][y_test_rgb.index[3*i]]*255))))
    plt.title('diff: '+str(round(diff_color.loc[diff_color.index[3*i]][0],3))+'\npredicted Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(184)
    plt.imshow(Image.new("RGB", (50,50),(int(y_test_rgb['r'][y_test_rgb.index[3*i+1]]*255),int(y_test_rgb['g'][y_test_rgb.index[3*i+1]]*255),int(y_test_rgb['b'][y_test_rgb.index[3*i+1]]*255))))
    plt.title('index: '+str(y_test_rgb.index[3*i+1])+'\nReal Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(185)
    plt.imshow(Image.new("RGB", (50,50),(int(pred_rgb['r'][y_test_rgb.index[3*i+1]]*255),int(pred_rgb['g'][y_test_rgb.index[3*i+1]]*255),int(pred_rgb['b'][y_test_rgb.index[3*i+1]]*255))))
    plt.title('diff: '+str(round(diff_color.loc[diff_color.index[3*i+1]][0],3))+'\npredicted Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(187)
    plt.imshow(Image.new("RGB", (50,50),(int(y_test_rgb['r'][y_test_rgb.index[3*i+2]]*255),int(y_test_rgb['g'][y_test_rgb.index[3*i+2]]*255),int(y_test_rgb['b'][y_test_rgb.index[3*i+2]]*255))))
    plt.title('index: '+str(y_test_rgb.index[3*i+2])+'\nReal Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(188)
    plt.imshow(Image.new("RGB", (50,50),(int(pred_rgb['r'][y_test_rgb.index[3*i+2]]*255),int(pred_rgb['g'][y_test_rgb.index[3*i+2]]*255),int(pred_rgb['b'][y_test_rgb.index[3*i+2]]*255))))
    plt.title('diff: '+str(round(diff_color.loc[diff_color.index[3*i+2]][0],3))+'\npredicted Color')
    plt.xticks([])
    plt.yticks([])
    plt.show()